### 50. 文区切り
(. or ; or : or ? or !) → 空白文字 → 英大文字というパターンを文の区切りと見なし，入力された文書を1行1文の形式で出力せよ．

In [29]:
import re
def search(line, file):
    x = re.search('^(?P<sente>.*?)(?P<mark>[.;:?!])( (?P<newline>[A-Z].*))?$|^(?P<title>.*)$', line)
    if x:
        sente = x.group('sente')
        mark = x.group('mark')
        newline = x.group('newline')
        title = x.group('title')
        if mark:
            file.write(sente + mark + '\n')
        elif title:
            file.write(title + '\n')
        if newline:
            search(newline, file)
    #else:
        #print(line)
    
with open('data/file50', 'w'):
    pass
with open('data/file50', 'a') as a:
    for line in open('data/nlp.txt'):
        search(line, a)

### 51. 単語の切り出し
空白を単語の区切りとみなし，50の出力を入力として受け取り，1行1単語の形式で出力せよ．ただし，文の終端では空行を出力せよ．

In [31]:
def search(line, file):
    x = re.search('(?P<word>[^ ]*)( (?P<newline>.*))?', line)
    if x:
        word = x.group('word').strip('.,;:?!()\"')
        newline = x.group('newline')
        file.write(word + '\n')
        if newline:
            search(newline, file)
        else:
            file.write('\n')

with open('data/file51', 'w'):
    pass
with open('data/file51', 'a') as a:
    for line in open('data/file50'):
        search(line, a)

### 52. ステミング
51の出力を入力として受け取り，Porterのステミングアルゴリズムを適用し，単語と語幹をタブ区切り形式で出力せよ． Pythonでは，Porterのステミングアルゴリズムの実装としてstemmingモジュールを利用するとよい．

In [32]:
from stemming.porter2 import stem

with open('data/file52', 'w'):
    pass
with open('data/file52', 'a') as a:
    for line in open('data/file51'):
        line = line.strip('\n')
        a.write(line + '\t' + stem(line) + '\n')

### 53. Tokenization
Stanford Core NLPを用い，入力テキストの解析結果をXML形式で得よ．また，このXMLファイルを読み込み，入力テキストを1行1単語の形式で出力せよ．

In [1]:
import re

In [4]:
with open('data/file53', 'w'):
    pass
with open('data/file53', 'a') as a:
    for line in  open('data/nlp.txt.xml'):
        x = re.search('\<word\>(?P<word>.*?)\<\/word\>', line)
        if x:
            a.write(x.group('word') + '\n')

### 54. 品詞タグ付け
Stanford Core NLPの解析結果XMLを読み込み，単語，レンマ，品詞をタブ区切り形式で出力せよ．

In [12]:
with open('data/file54', 'w'):
    pass
with open('data/file54', 'a') as a:
    for line in  open('data/nlp.txt.xml'):
        x = re.search('\<(?P<title>.*?)\>(?P<info>.*?)\<\/(?P=title)\>', line)
        if x:
            title = x.group('title')
            info = x.group('info')
            if title == 'word' or title == 'lemma':
                a.write(info + '\t')
            elif title == 'POS':
                a.write(info + '\n')

### 55. 固有表現抽出
入力文中の人名をすべて抜き出せ．

In [23]:
with open('data/file55', 'w'):
    pass
with open('data/file55', 'a') as a:
    flag = False
    stack = []
    for line in  open('data/nlp.txt.xml'):
        x = re.search('\<(?P<title>.*?)\>(?P<info>.*?)\<\/(?P=title)\>', line)
        if x:
            title = x.group('title')
            info = x.group('info')
            if title == 'word':
                stack.append(info)
            elif title == 'NER' and info != 'PERSON' and not flag:
                stack = []
            elif title == 'NER' and info != 'PERSON' and flag:
                a.write('\t'.join(stack[:-1]) + '\n')
                flag = False
                stack = []
            elif title == 'NER' and info == 'PERSON':
                flag = True

### 56. 共参照解析
Stanford Core NLPの共参照解析の結果に基づき，文中の参照表現（mention）を代表参照表現（representative mention）に置換せよ．ただし，置換するときは，「代表参照表現（参照表現）」のように，元の参照表現が分かるように配慮せよ．

In [172]:
flag1 = False
flag2 = False
coreference = []
mentions = []
mention = {}
for line in open('data/nlp.txt.xml'):
    x = re.search('\<coreference\>', line)
    if x and not flag1:
        flag1 = True
        continue
    if x and flag1:
        flag2 = True
        continue
    x = re.search('\<\/coreference\>', line)
    if x and not flag2:
        flag1 = False
        continue
    if x and flag2:
        flag2 = False
        if len(mentions) > 1:
            coreference.append(mentions)
        mentions = []
        continue
    x = re.search('\<(?P<title>.*?)\>(?P<info>.*?)\<\/(?P=title)\>', line)
    if x and flag2:
        title = x.group('title')
        info = x.group('info')
        if title == 'text' and mentions != [] and info == mentions[0]['text']:
            continue
        if title == 'sentence' or title == 'start' or title =='end':
            mention.update({title: int(info) - 1})
        elif title == 'text':
            mention.update({title:info})
            mentions.append(mention)
            mention = {}

In [173]:
[[mention['text'] for mention in mentions] for mentions in coreference]

[['many speech recognition systems',
  'These systems',
  'these systems',
  'the systems'],
 ['machine learning algorithms',
  'These algorithms',
  'the earliest-used algorithms'],
 ['The machine-learning paradigm', 'The paradigm of machine learning'],
 ['you', 'your'],
 ['Such models', 'they'],
 ['My head', 'your head'],
 ['NLP', 'its'],
 ['hand-written rules --', 'the rules']]

In [174]:
sentences = []
sentence= []
for line in open('data/nlp.txt.xml'):
    x = re.search('\<word\>(?P<word>.*?)\<\/word\>', line)
    if x:
        sentence.append(x.group('word'))
    elif re.search('\<\/sentence\>', line):
        sentences.append(sentence)
        sentence = []

In [175]:
import copy
new_sentences = copy.deepcopy(sentences)
for mentions in coreference:
    for mention in mentions[1:]:
        old_text = new_sentences[mention['sentence']][mention['start'] : mention['end']]
        if len(old_text) == 1:
            new_text = [mentions[0]['text'] + '(' + old_text[0] + ')']
        else:
            new_text = [mentions[0]['text'] + '(' + old_text[0]] + old_text[1:-1] + [old_text[-1] + ')']
        new_sentences[mention['sentence']][mention['start'] : mention['end']] = new_text

In [176]:
new_sentences[11][34:40]

['say', 'My head(you(your)', 'head)', 'hurts', '?', "''"]

In [177]:
sentences[11][34:40]

['say', 'your', 'head', 'hurts', '?', "''"]

In [178]:
with open('data/file56', 'w'):
    pass
with open('data/file56', 'a') as a:
    for sentence in new_sentences:
        a.write(' '.join(sentence) + '\n')